<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random

from tensorflow.keras.layers import Dense, Input,InputLayer,Flatten,Activation,LSTM,SimpleRNN,GRU,TimeDistributed
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

[]

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

In [ ]:
def load_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    test_file_path = path + "te.translit.sampled.test.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    

    #File containing training data
    with open(train_file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    #storing a list of list containing the two pair of words
    train_pairs = []
    for line in lines:
        train_pairs.append(line.strip("\n").split("\t"))

    #File containing test data
    with open(test_file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    #storing a list of list containing the two pair of words
    test_pairs = []
    for line in lines:
        test_pairs.append(line.strip("\n").split("\t"))

    #File containing validation data
    with open(val_file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    #storing a list of list containing the two pair of words
    val_pairs = []
    for line in lines:
        val_pairs.append(line.strip("\n").split("\t"))

    return train_pairs, val_pairs, test_pairs


In [ ]:
train_pairs, val_pairs, test_pairs = load_data()

#print(train_pairs)

In [ ]:
def prepare_data(dataset):
    
    input_words = []
    target_words = []
    input_tokens = set()
    target_tokens = set()

    #iterate for each pair of words in the dataset
    for pair in dataset:

        #Add these words to the input and target list respectively
        input_words.append(pair[1])
        target_words.append("\t"+pair[0]+"\n")

        #adding the characters of the word into the tokens set to store the 
        #list of all the characters
        for char in pair[1]:
            input_tokens.add(char)
        for char in pair[0]:
            target_tokens.add(char)

    input_tokens = sorted(list(input_tokens) + [" "])
    target_tokens = sorted(list(target_tokens) + [" ", '\t', '\n'])

    num_enc_tokens = len(input_tokens)
    num_tar_tokens = len(target_tokens)

    max_inp_len = max([len(inp) for inp in input_words])
    max_tar_len = max([len(inp) for inp in target_words])

    print("Number of samples:", len(dataset))
    print("Number of unique input tokens:", num_enc_tokens)
    print("Number of unique output tokens:", num_tar_tokens)
    print("Max sequence length for inputs:", max_inp_len)
    print("Max sequence length for outputs:", max_tar_len)

    inp_token_index = dict([(char,i) for i, char in enumerate(input_tokens)])
    tar_token_index = dict([(char,i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in inp_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in tar_token_index.items())

    return input_words, target_words, inp_token_index, tar_token_index, reverse_input_char_index, reverse_target_char_index




In [ ]:
input_words, target_words, inp_token_index, tar_token_index, reverse_input_char_index, reverse_target_char_index = prepare_data(train_pairs)

Number of samples: 58550
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 25
Max sequence length for outputs: 22


In [ ]:
print(inp_token_index)

{'\t': 0, '\n': 1, ' ': 2, 'ం': 3, 'ః': 4, 'అ': 5, 'ఆ': 6, 'ఇ': 7, 'ఈ': 8, 'ఉ': 9, 'ఊ': 10, 'ఋ': 11, 'ఎ': 12, 'ఏ': 13, 'ఐ': 14, 'ఒ': 15, 'ఓ': 16, 'ఔ': 17, 'క': 18, 'ఖ': 19, 'గ': 20, 'ఘ': 21, 'చ': 22, 'ఛ': 23, 'జ': 24, 'ఝ': 25, 'ఞ': 26, 'ట': 27, 'ఠ': 28, 'డ': 29, 'ఢ': 30, 'ణ': 31, 'త': 32, 'థ': 33, 'ద': 34, 'ధ': 35, 'న': 36, 'ప': 37, 'ఫ': 38, 'బ': 39, 'భ': 40, 'మ': 41, 'య': 42, 'ర': 43, 'ఱ': 44, 'ల': 45, 'ళ': 46, 'వ': 47, 'శ': 48, 'ష': 49, 'స': 50, 'హ': 51, 'ా': 52, 'ి': 53, 'ీ': 54, 'ు': 55, 'ూ': 56, 'ృ': 57, 'ె': 58, 'ే': 59, 'ై': 60, 'ొ': 61, 'ో': 62, 'ౌ': 63, '్': 64, '\u200c': 65}


In [ ]:
def one_hot_encoding(input_words, target_words, inp_index, tar_index):

    max_inp_len = max([len(inp) for inp in input_words])
    max_tar_len = max([len(inp) for inp in target_words])

    encoder_input_data = np.zeros(
        (len(input_words), max_inp_len, len(inp_index)), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input_words), max_tar_len, len(tar_index)), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input_words), max_tar_len, len(tar_index)), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input_words, target_words)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t,inp_index[char]] = 1.0
        encoder_input_data[i, t+1:, inp_index[" "]] = 1.0

        for t, char in enumerate(tar):
            decoder_input_data[i,t,tar_index[char]] = 1.0

            if t>0:
                decoder_target_data[i,t-1,tar_index[char]] = 1.0

        decoder_input_data[i, t+1:, tar_index[" "]] = 1.0
        decoder_target_data[i, t:, tar_index[" "]] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data


In [ ]:
encoder_input_data, decoder_input_data, decoder_target_data =one_hot_encoding(input_words, target_words, inp_token_index, tar_token_index)

In [ ]:
max_inp_len = max([len(inp) for inp in input_words])
max_tar_len = max([len(inp) for inp in target_words])
num_enc_tokens = len(inp_token_index)
num_dec_tokens = len(tar_token_index)

In [ ]:
def rnn(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder

  enc_in = Input(shape=(None,))
  enc_out = Embedding(num_enc_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  for i in range(1,num_encoders+1):
    encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    enc_layers.append(encoder)
    enc_out, state_h = encoder(enc_out)
    enc_states.append([state_h])
    

  dec_in = Input(shape=(None, ))
  dec_out = Embedding(num_dec_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []
  for i in range(1,num_decoders+1):
    decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    dec_layers.append(decoder)
    dec_out, _ = decoder(dec_out, initial_state = enc_states[0])
  
  
  dec_dense = Dense(num_tar_tokens, activation="softmax")
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def lstm(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
  enc_in = Input(shape=(None,))
  enc_out = Embedding(num_enc_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  for i in range(1,num_encoders+1):
    encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    enc_layers.append(encoder)
    enc_out, state_h, state_c = encoder(enc_out)
    enc_states.append([state_h, state_c])
    

  dec_in = Input(shape=(None, ))
  dec_out = Embedding(num_dec_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []
  for i in range(1,num_decoders+1):
    decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    dec_layers.append(decoder)
    dec_out, _, _ = decoder(dec_out, initial_state = enc_states[0])
  
  
  dec_dense = Dense(num_tar_tokens, activation="softmax")
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def gru(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
 
  enc_in = Input(shape=(None,))
  enc_out = Embedding(num_enc_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  for i in range(1,num_encoders+1):
    encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    enc_layers.append(encoder)
    enc_out, state_h = encoder(enc_out)
    enc_states.append([state_h])
    

  dec_in = Input(shape=(None, ))
  dec_out = Embedding(num_dec_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []
  for i in range(1,num_decoders+1):
    decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    dec_layers.append(decoder)
    dec_out, _ = decoder(dec_out, initial_state = enc_states[0])
  
  
  dec_dense = Dense(num_tar_tokens, activation="softmax")
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def transliteration(config,inp_charint,tgt_charint):
  num_encoders=config["num_encoders"]
  cell=config["cell"]
  embed_size=config["embed_size"]
  dropout=config["dropout"]
  num_decoders=config["num_decoders"]
  hidden_layer_size=config["hidden_layer_size"]

  if cell == "RNN":
    model, enc_layers, dec_layers=rnn(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model
  elif cell == "LSTM":
    model, enc_layers, dec_layers=lstm(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model
  elif cell == "GRU":
    model, enc_layers, dec_layers=gru(num_enc_tokens, num_tar_tokens, max_inp_len, max_tar_len, num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model
  


In [ ]:
def inference(model, num_encoders, num_decoders, enc_layers, dec_layers, cell, hidden_layer_size):

    enc_in = model.input[0]
    EmbeddingLayer = model.layers[2]    
    enc_out = EmbeddingLayer(enc_in)

    enc_states = []
    if cell == 'RNN':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 
    elif cell == 'LSTM':
        for i in range(num_encoders):
            enc_out, state_h, state_c = enc_layers[i](enc_out)
            enc_states += [state_h, state_c]   
    elif cell == 'RNN':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 

    encoder = Model(enc_in, enc_states + [enc_out])


    dec_in = model.input[1]    
    EmbeddingLayer = model.layers[3] 
    dec_out = EmbeddingLayer(dec_in)

    dec_states = []
    dec_initial_states = []
    
    if cell == 'RNN' :
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,)]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]
    elif cell == "LSTM":
        j=0
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size, )) , Input(shape=(hidden_layer_size, ))]
            dec_out, state_h, state_c = dec_layers[i](dec_out, initial_state=dec_initial_states[i+j:i+j+2])
            dec_states += [state_h , state_c]
            j += 1
    elif cell == "GRU":
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,)]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]

    dec_dense = model.layers[4 + 2*num_encoders]
    dec_out = dec_dense(dec_out)
    decoder = Model([dec_in] + dec_initial_states, [dec_out] + dec_states)

    return encoder, decoder 

In [ ]:
def decode_sequence(input_seq, encoder, decoder):

    states_value = encoder.predict(input_seq)
    states_value = states_value[:-1]

    target_seq = np.zeros((1, 1)) 
    target_seq[0, 0] = tar_token_index["\t"]

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = output_tokens[1:]
        
    return decoded_sentence

In [ ]:
# Enter the project and the entity name for wandb sweeps
project_name=""
entity_name=""


In [ ]:
def train():
  default_config={
      "cell": "RNN",
      "embed_size":256,
      "dropout":0.2,
      "num_encoders": 1,
      "num_decoders": 1,
      "hidden_layer_size":128,
      "epochs": 1,
      "batch_size": 64
  }

  wandb.init(config=default_config,project=project_name,entity=entity_name)
  config=wandb.config
  wandb.run.name=(
      str(config.cell)
      +"_"+str(config.embed_size)
      +"_"+str(config.dropout)
      +"_"+str(config.num_encoders)
      +"_"+str(config.num_decoders)
      +"_"+str(config.hidden_layer_size)
      +"_"+str(config.epochs)
      +"_"+str(config.batch_size)
  )

  wandb.run.save()

  model=transliteration(config,inp_token_index,tgt_token_index)
  
  model.compile(
      optimizer="adam",
      loss="categorical_crossentropy",
      metrics=["accuracy"]
  )
  earlystopping = EarlyStopping(
          monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
      )

  model.fit(
      [e_in,d_in],
      d_t,
      batch_size=config.batch_size,
      epochs=config.epochs,
      callbacks=[earlystopping,WandbCallback()]
  )

  wandb.finish()
  

In [ ]:
def sweeper(project_name,entity_name)
  sweep_configuration={
      "method": "bayes",
      "metric": {"name": "val_accuracy", "goal": "maximize"},
      "parameters":{
          "cell":{"values":["RNN","GRU","LSTM"]},
          "embed_size":{"values":[16,32,64,256]},
          "hidden_layer_size":{"values":[16,32,64,256]},
          "num_encoders":{"values":[1,2,3]},
          "num_decoders":{"values":[1,2,3]},
          "dropout":{"values":[0.2,0.3]},
          "epochs":{"values":[5,10,15,20]},
          "batch_size":{"values":[32,64]}
      }
  }

  sweep_id=wandb.sweep(sweep_configuration,project=project_name,entity=entity_name)
  wandb.agent(sweep_id,train)


In [ ]:
# Running the sweeps for hyperparameter tuning
sweeper(project_name,entity_name)